# Quantitative Momentum Strategy
"Momentum investing" means investing in the stocks that have increased in price the most.
For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_excel('sp_500_stocks.xlsx')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2301880574125,
 'week52high': 141.62,
 'week52low': 56.58,
 'week52change': 0.8521717406515815,
 'sharesOutstanding': 17087027394,
 'float': 0,
 'avg10Volume': 111685091,
 'avg30Volume': 109154714,
 'day200MovingAvg': 115.58,
 'day50MovingAvg': 129.11,
 'employees': 0,
 'ttmEPS': 3.41,
 'ttmDividendRate': 0.812055638095389,
 'dividendYield': 0.00610011990011286,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-31',
 'nextEarningsDate': '0',
 'peRatio': 41.4562223925185,
 'beta': 1.1995595192223403,
 'maxChangePercent': 52.44021888034614,
 'year5ChangePercent': 4.52170111132889,
 'year2ChangePercent': 2.5218922054063366,
 'year1ChangePercent': 0.8901323103159958,
 'ytdChangePercent': 0.8493080489010567,
 'month6ChangePercent': 0.5059768983289873,
 'month3ChangePercent': 0.1741986971703687,
 'month1ChangePercent': 0.1506812955029435,
 'day30ChangePercent': 0.12498185743526623,
 'day5ChangePercent': 0.021578204649067664}

## Parsing Our API Call
This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8901323103159958

## Executing A Batch API Call & Building Our DataFrame
Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    '''Yield successive n-sized chunks from lst/'''
    for i in range(0, len(lst), n):
        yield lst[i:i +n]

symbol_group = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_group)):
    symbol_strings.append(','.join(symbol_group[i]))

columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series([symbol, 
                   data[symbol]['quote']['latestPrice'],
                   data[symbol]['stats']['year1ChangePercent'],
                   'N/A'], 
                  index = columns), 
                  ignore_index = True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,120.23,0.393743,N/A
1,AAL,16.76,-0.428059,N/A
2,AAP,161.91,-0.001506,N/A
3,AAPL,137.50,0.887252,N/A
4,ABBV,105.74,0.255889,N/A
...,...,...,...,...
500,YUM,112.22,0.113886,N/A
501,ZBH,157.92,0.032547,N/A
502,ZBRA,389.78,0.528865,N/A
503,ZION,43.31,-0.136116,N/A


## Removing Low-Momentum Stocks
The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,39.30,2.183614,N/A
1,NVDA,536.81,1.310981,N/A
2,PYPL,236.77,1.193321,N/A
3,LB,38.23,1.178216,N/A
4,ALB,153.73,1.158999,N/A
5,FCX,27.00,1.074538,N/A
6,AMD,94.37,1.067714,N/A
7,ABMD,338.47,0.980400,N/A
8,CDNS,140.80,0.967529,N/A
9,NOW,553.16,0.948566,N/A


## Calculating the Number of Shares to Buy
Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,39.30,2.183614,5089
1,NVDA,536.81,1.310981,372
2,PYPL,236.77,1.193321,844
3,LB,38.23,1.178216,5231
4,ALB,153.73,1.158999,1300
5,FCX,27.00,1.074538,7407
6,AMD,94.37,1.067714,2119
7,ABMD,338.47,0.980400,590
8,CDNS,140.80,0.967529,1420
9,NOW,553.16,0.948566,361


## Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:
- High-quality momentum stocks show "slow and steady" outperformance over long periods of time
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:
- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns
Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([symbol,
                       data[symbol]['quote']['latestPrice'],
                       'N/A',
                       data[symbol]['stats']['year1ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month6ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month3ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month1ChangePercent'],
                       'N/A',
                       'N/A'
                       ],
                      index=hqm_columns),
            ignore_index=True)
        
hqm_dataframe   

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.40,N/A,0.392406,N/A,0.357356,N/A,0.178450,N/A,0.028941,N/A,N/A
1,AAL,16.52,N/A,-0.428302,N/A,0.216724,N/A,0.318782,N/A,0.080297,N/A,N/A
2,AAP,160.57,N/A,-0.001514,N/A,0.113178,N/A,0.041116,N/A,0.054113,N/A,N/A
3,AAPL,139.24,N/A,0.860905,N/A,0.485491,N/A,0.177251,N/A,0.151238,N/A,N/A
4,ABBV,106.44,N/A,0.258047,N/A,0.123303,N/A,0.235562,N/A,0.003759,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.08,N/A,0.109293,N/A,0.279527,N/A,0.220380,N/A,0.022742,N/A,N/A
501,ZBH,159.48,N/A,0.031867,N/A,0.319311,N/A,0.147869,N/A,0.016627,N/A,N/A
502,ZBRA,400.24,N/A,0.514089,N/A,0.530187,N/A,0.496795,N/A,0.018609,N/A,N/A
503,ZION,44.18,N/A,-0.135993,N/A,0.337439,N/A,0.515104,N/A,0.098527,N/A,N/A


## Calculating Momentum Percentiles
We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:
- One-Year Price Return
- Six-Month Price Return
- Three-Month Price Return
- One-Month Price Return

Here's how we'll do this:

In [11]:
# Replace N/A values in DataFrame with zero
hqm_dataframe.replace(to_replace='N/A', value=0, inplace=True)

time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

#Print the entire DataFrame    
hqm_dataframe

/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:1942: RuntimeWarning: invalid value encountered in less
  left = np.count_nonzero(a < score)
/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:1943: RuntimeWarning: invalid value encountered in less_equal
  right = np.count_nonzero(a <= score)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.40,0,0.392406,0.853465,0.357356,0.673267,0.178450,0.518812,0.028941,0.598020,0
1,AAL,16.52,0,-0.428302,0.029703,0.216724,0.425743,0.318782,0.778218,0.080297,0.829703,0
2,AAP,160.57,0,-0.001514,0.374257,0.113178,0.247525,0.041116,0.217822,0.054113,0.742574,0
3,AAPL,139.24,0,0.860905,0.964356,0.485491,0.833663,0.177251,0.516832,0.151238,0.958416,0
4,ABBV,106.44,0,0.258047,0.724752,0.123303,0.259406,0.235562,0.635644,0.003759,0.405941,0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.08,0,0.109293,0.530693,0.279527,0.538614,0.220380,0.603960,0.022742,0.560396,0
501,ZBH,159.48,0,0.031867,0.417822,0.319311,0.607921,0.147869,0.445545,0.016627,0.504950,0
502,ZBRA,400.24,0,0.514089,0.912871,0.530187,0.879208,0.496795,0.932673,0.018609,0.526733,0
503,ZION,44.18,0,-0.135993,0.200000,0.337439,0.651485,0.515104,0.938614,0.098527,0.885149,0


## Calculating the HQM Score
We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks
As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,26.76,0,1.078870,0.982178,1.449104,0.990099,0.755017,0.982178,0.133406,0.940594,0.973762
1,ALGN,550.18,0,0.936162,0.974257,0.994670,0.982178,0.650458,0.976238,0.102683,0.895050,0.956931
2,ALB,152.16,0,1.111635,0.984158,0.996658,0.984158,0.748017,0.980198,0.087454,0.855446,0.950990
3,SIVB,398.87,0,0.558924,0.922772,0.882386,0.974257,0.634882,0.962376,0.085773,0.849505,0.927228
4,DIS,185.55,0,0.263185,0.738614,0.626840,0.932673,0.457318,0.912871,0.240652,0.986139,0.892574
5,LRCX,504.47,0,0.710748,0.944554,0.592593,0.916832,0.450142,0.906931,0.066493,0.788119,0.889109
6,PAYC,455.88,0,0.735975,0.950495,0.498541,0.853465,0.475705,0.926733,0.074965,0.815842,0.886634
7,APTV,136.52,0,0.391446,0.849505,0.759418,0.958416,0.433432,0.893069,0.076157,0.817822,0.879703
8,ALXN,160.43,0,0.473260,0.902970,0.407180,0.742574,0.404039,0.877228,0.269421,0.990099,0.878218
9,TWTR,54.86,0,0.727481,0.946535,0.882676,0.976238,0.223542,0.613861,0.170626,0.968317,0.876238


## Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:10000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,26.76,7473,1.078870,0.982178,1.449104,0.990099,0.755017,0.982178,0.133406,0.940594,0.973762
1,ALGN,550.18,363,0.936162,0.974257,0.994670,0.982178,0.650458,0.976238,0.102683,0.895050,0.956931
2,ALB,152.16,1314,1.111635,0.984158,0.996658,0.984158,0.748017,0.980198,0.087454,0.855446,0.950990
3,SIVB,398.87,501,0.558924,0.922772,0.882386,0.974257,0.634882,0.962376,0.085773,0.849505,0.927228
4,DIS,185.55,1077,0.263185,0.738614,0.626840,0.932673,0.457318,0.912871,0.240652,0.986139,0.892574
5,LRCX,504.47,396,0.710748,0.944554,0.592593,0.916832,0.450142,0.906931,0.066493,0.788119,0.889109
6,PAYC,455.88,438,0.735975,0.950495,0.498541,0.853465,0.475705,0.926733,0.074965,0.815842,0.886634
7,APTV,136.52,1464,0.391446,0.849505,0.759418,0.958416,0.433432,0.893069,0.076157,0.817822,0.879703
8,ALXN,160.43,1246,0.473260,0.902970,0.407180,0.742574,0.404039,0.877228,0.269421,0.990099,0.878218
9,TWTR,54.86,3645,0.727481,0.946535,0.882676,0.976238,0.223542,0.613861,0.170626,0.968317,0.876238


## Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.
XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:
- String format for tickers
- \$XX.XX format for stock prices
- \$XX,XXX format for market capitalization
- Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [19]:
writer.save()